In [4]:
import numpy as np
import pandas as pd


In [ ]:
data = pd.read_csv('/Users/hj/projects/Aimers/data/raw/train.csv')


,ID,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,...,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,임신 성공 여부
0,TRAIN_000000,TRZKPL,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0
1,TRAIN_000001,TRYBLT,만45-50세,NaN,IVF,ICSI,0,알 수 없음,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0
2,TRAIN_000002,TRVNRY,만18-34세,NaN,IVF,IVF,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,2.0,NaN,0
3,TRAIN_000003,TRJXFG,만35-37세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0
4,TRAIN_000004,TRVNRY,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0


In [11]:
data.head()
y = data['임신 성공 여부']

In [15]:
data.shape # (256351, 69)

(256351, 69)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 69 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256351 non-null  object 
 1   시술 시기 코드               256351 non-null  object 
 2   시술 당시 나이               256351 non-null  object 
 3   임신 시도 또는 마지막 임신 경과 연수  9370 non-null    float64
 4   시술 유형                  256351 non-null  object 
 5   특정 시술 유형               256349 non-null  object 
 6   배란 자극 여부               256351 non-null  int64  
 7   배란 유도 유형               256351 non-null  object 
 8   단일 배아 이식 여부            250060 non-null  float64
 9   착상 전 유전 검사 사용 여부       2718 non-null    float64
 10  착상 전 유전 진단 사용 여부       250060 non-null  float64
 11  남성 주 불임 원인             256351 non-null  int64  
 12  남성 부 불임 원인             256351 non-null  int64  
 13  여성 주 불임 원인             256351 non-null  int64  
 14  여성 부 불임 원인             256351 non-nu

In [16]:
# 결측치 
print(data.isnull().sum()) # 결측을 보는 다른 방법 추가하기

ID                            0
시술 시기 코드                      0
시술 당시 나이                      0
임신 시도 또는 마지막 임신 경과 연수    246981
시술 유형                         0
                          ...  
난자 해동 경과일                254915
난자 혼합 경과일                 53735
배아 이식 경과일                 43566
배아 해동 경과일                215982
임신 성공 여부                      0
Length: 69, dtype: int64


In [23]:
# 상관관계 확인 
num_data = data.select_dtypes('int')
corr_m = num_data.corr()
corr_m['임신 성공 여부'].sort_values()

부부 주 불임 원인           -0.016099
여성 부 불임 원인           -0.016034
남성 부 불임 원인           -0.016025
여성 주 불임 원인           -0.015945
부부 부 불임 원인           -0.015346
남성 주 불임 원인           -0.015020
불임 원인 - 정자 운동성       -0.003693
불임 원인 - 자궁경부 문제      -0.003686
불임 원인 - 자궁내막증        -0.002226
불임 원인 - 정자 형태        -0.001866
불임 원인 - 정자 면역학적 요인   -0.001166
불임 원인 - 정자 농도        -0.000626
불임 원인 - 난관 질환        -0.000544
불명확 불임 원인             0.001473
불임 원인 - 배란 장애         0.025021
배란 자극 여부              0.033212
불임 원인 - 남성 요인         0.037908
임신 성공 여부              1.000000
불임 원인 - 여성 요인              NaN
Name: 임신 성공 여부, dtype: float64

In [ ]:
# 범주형 변수 EDA 
cate_cols = data.select_dtypes('object')
print(cate_cols.columns)

for col in cate_cols :
    print(data[col].nunique())  # ID 제외, 

Index(['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 유도 유형',
       '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수',
       '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수',
       '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이'],
      dtype='object')
256351
7
7
2
24
4
13
7
7
7
7
7
7
6
7
6
5
3
4
5
7


In [28]:
# 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

drop_cols = ['특정 시술 유형', '배아 생성 주요 이유', 'ID']
df = data.drop(columns = drop_cols)

In [33]:
X = df.drop('임신 성공 여부', axis=1)
y = df['임신 성공 여부']

In [34]:
from sklearn.impute import SimpleImputer
# 5. 결측치 처리
# 숫자형과 범주형 칼럼 분리
numeric_cols = X.select_dtypes(include=['int', 'float']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# 숫자형 변수: 중앙값으로 대체
imputer_num = SimpleImputer(strategy='median')
X[numeric_cols] = imputer_num.fit_transform(X[numeric_cols])

# 범주형 변수: 최빈값으로 대체
imputer_cat = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = imputer_cat.fit_transform(X[categorical_cols])

In [36]:
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    label_encoders[col] = le

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
rf = RandomForestClassifier(n_estimators=100, random_state= 73)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=73)

In [38]:
# 9. 테스트 데이터 예측 및 평가
y_pred = rf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.91      0.83     37741
           1       0.44      0.20      0.27     13530

    accuracy                           0.72     51271
   macro avg       0.60      0.55      0.55     51271
weighted avg       0.67      0.72      0.68     51271



In [40]:
# 제출 데이터 만들기
test = pd.read_csv('/Users/hj/projects/Aimers/data/raw/test.csv')

In [41]:
if 'ID' in test.columns:
    test_ids = test['ID']
else:
    test_ids = pd.Series(range(len(test)), name="ID")

In [42]:
test = test.drop(columns=drop_cols, errors='ignore')

In [43]:
# 훈련 시 사용한 숫자형, 범주형 칼럼 확인 (테스트 데이터에도 동일한 칼럼명이 있어야 함)
test_numeric_cols = test.select_dtypes(include=['int', 'float']).columns
test_categorical_cols = test.select_dtypes(include=['object']).columns

# 1) 숫자형 변수: 훈련 시 학습한 imputer_num 적용
test[test_numeric_cols] = imputer_num.transform(test[test_numeric_cols])

# 2) 범주형 변수: 훈련 시 학습한 imputer_cat 적용
test[test_categorical_cols] = imputer_cat.transform(test[test_categorical_cols])

# 3) 범주형 변수: 훈련 시 저장한 LabelEncoder 적용  
for col in test_categorical_cols:
    if col in label_encoders:
        le = label_encoders[col]
        # 테스트 데이터에 새로운 범주가 있을 경우, 해당 값을 -1로 처리
        test[col] = test[col].astype(str)
        test[col] = test[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
    else:
        # 만약 해당 칼럼에 대해 인코더가 없으면 pass (원래 없었어야 함)
        pass

# -------------------------------
# 4. 테스트 데이터에 대해 예측 수행
# -------------------------------
predictions = rf.predict(test)

# (필요에 따라, 타겟 변수에 원래 문자열 레이블을 복원하려면 아래 주석을 해제)
# predictions = le_target.inverse_transform(predictions)

# -------------------------------
# 5. sample_submission.csv 파일 생성
# -------------------------------
submission = pd.DataFrame({
    'ID': test_ids,
    '임신성공여부': predictions
})

submission.to_csv("sample_submission.csv", index=False)
print("sample_submission.csv 파일이 생성되었습니다.")

NameError: name 'label_encoders' is not defined